# Load Projection Matrix

In [3]:
import numpy as np

# open csv file and organize the 3 rows of 4 columns into a 3x4 matrix
matrix = np.genfromtxt('../../data/projection_matrix.csv', delimiter=',')

# Separate Intrinsic and Extrinsic Matrices (#TODO)

In [66]:
H = matrix[0:3, 0:3]
H_inv = np.linalg.inv(H)
h = matrix[0:3, 3]
X0 = np.dot(-H_inv, h)
print("X0: ", X0)
# real is [0, 0.35, 0.7]

# QR decomposition
R, Q = np.linalg.qr(H)
R = -R
print("R: ", R)
ang = np.arccos(R[1, 1])
print("ang: ", ang * 180 / np.pi)

ang = -42.97 * np.pi / 180
T = np.array([
    [1, 0, 0, 0],
    [0, np.cos(ang), -np.sin(ang), 0.35],
    [0, np.sin(ang), np.cos(ang), 0.7],
    [0, 0, 0, 1]
])
print("T: ", T)

fx = matrix[0, 0]
a1 = T[1, 1]
a2 = T[1, 2]
b1 = T[2, 1]
b2 = T[2, 2]
c = matrix[1, 1]
d = matrix[1, 2]
fy = (c*b2/b1 - d) / (a1*b2/b1 - a2)
print("fx: ", fx)
print("fy: ", fy)
K = np.array([
    [fx, 0, 0],
    [0, fy, 0],
    [0, 0, 1]
])
K = np.hstack((K, np.zeros((3, 1))))
P = np.dot(K, T)
print("P: ", P/0.7)

X0:  [-2.72301637e-05  3.27839947e-01  8.26040676e-01]
R:  [[ 0.99975094 -0.01976936 -0.01035482]
 [ 0.00622818  0.69270933 -0.72118998]
 [-0.02143034 -0.72094587 -0.69265994]]
ang:  46.155039317830635
T:  [[ 1.          0.          0.          0.        ]
 [ 0.          0.73171069  0.68161533  0.35      ]
 [ 0.         -0.68161533  0.73171069  0.7       ]
 [ 0.          0.          0.          1.        ]]
fx:  2.048609161567175
fy:  -3.7603323745396056
P:  [[ 2.92658452  0.          0.          0.        ]
 [ 0.         -3.93067916 -3.66157171 -1.88016619]
 [ 0.         -0.97373619  1.04530099  1.        ]]


# Direct Transformation (World -> Pixel)

In [4]:
px = np.dot(matrix, np.array([-1.5, 2.5, 0, 1]))
# print(px[0] * 960 + 960/2, px[1] * 528 + 528/2)
px = px / px[2]
print(px[0] * 960 + 960/2, px[1] * 528 + 528/2)

90.25179592606435 46.87221119401002


# Inverse Transformation (Pixel -> World with z=0)

In [25]:
import time

x = 173
y = 65
px = np.array([(x - 960/2) / 960, (y - 528/2) / 528, 1])

M = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 0],
    [0, 0, 1]]
)
t0 = time.time()
inv_matrix = np.dot(M, np.linalg.inv(np.dot(matrix, M)))
result = np.dot(inv_matrix, px)
result = result / result[-1]
t1 = time.time()
print(f"Took {(t1 - t0) * 1000:.2f}ms to compute the result")
print(result)

Took 0.73ms to compute the result
[-1.10914892  2.32785784  0.          1.        ]
